In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xml
import xml.etree.ElementTree as ET
import cv2
import math
import warnings
from skimage.transform import resize
import os
from scipy import ndimage
import shutil

In [ ]:
# see if the noetbook accesses the GPU
tf.config.list_physical_devices("GPU")

##### Preprocessing for train data

In [ ]:
# Train data locations
block_0101 = '../Block_1_TN/Block_1_images_and_xml'
block_0102 = '../Block_2_TN/Block_2_images_and_xml'
block_0203 = '../Block_9_TN/Block_9_images_and_xml'
block_0301 = '../Block_13_TN/Block_13_images_and_xml'

train_blocks = [block_0101, block_0102, block_0203, block_0301]

# valid data location
block_0204 = '../Block_10_TN/Block_10_images_and_xml'

valid_blocks = [block_0204]


# test data location
block_0103 = '../Block_3_TN/Block_3_images_and_xml'
block_0104 = '../Block_4_TN/Block_4_images_and_xml'
block_0105 = '../Block_5_TN/Block_5_images_and_xml'
block_0106 = '../Block_6_TN/Block_6_images_and_xml'
block_0201 = '../Block_7_TN/Block_7_images_and_xml'
block_0202 = '../Block_8_TN/Block_8_images_and_xml'
block_0205 = '../Block_11_TN/Block_11_images_and_xml'
block_0206 = '../Block_12_TN/Block_12_images_and_xml'
block_0302 = '../Block_14_TN/Block_14_images_and_xml'
block_0303 = '../Block_15_TN/Block_15_images_and_xml'
block_0304 = '../Block_16_TN/Block_16_images_and_xml'
block_0305 = '../Block_17_TN/Block_17_images_and_xml'
block_0306 = '../Block_18_TN/Block_18_images_and_xml'

test_blocks = [block_0103, block_0104, block_0105, block_0106, block_0201, block_0202, block_0205, block_0206, block_0302, block_0303, block_0304, block_0305, block_0306]

In [ ]:
# list all the image and the xml files for the train data
all_train_contents = []
for item in train_blocks:
    block_contents = os.listdir(item)
    block_contents.sort()
    all_train_contents.append(block_contents)

In [ ]:
# need to separate the image and the xml files - define a function
def separate_img_xml(content_list):
    image_files = []
    xml_files = []
    for file in content_list:
        if file.split(".")[-1] == 'jpeg':
            image_files.append(file)
            image_files.sort()
        else:
            xml_files.append(file)
            xml_files.sort()
    return(image_files, xml_files)

In [ ]:
# separate the annotated and not annotated images
def separate_annot_and_not(img_list, xml_list):
    image_names = [name.split(".")[0] for name in xml_list]
    annotated_images = []
    not_annotated_images = []
    for file in img_list:
        if file.split('.')[0] in image_names:
            annotated_images.append(file)
        else:
            not_annotated_images.append(file)
    return(annotated_images, not_annotated_images)

In [ ]:
# separete the vertically and horizontally annotated images
def separate_annnot_vertical_horizontal(annotated_images, all_xml_files, n):
    annot_vertical_images = annotated_images[:n]
    annot_vertical_xmls = all_xml_files[:n]
    annot_horizontal_images = annotated_images[n:]
    annot_horizontal_xmls = all_xml_files[n:]
    return(annot_vertical_images, annot_vertical_xmls, annot_horizontal_images, annot_horizontal_xmls)

In [ ]:
# check if the annotated horizontal and vertical file names match
def check_annot(annot_vertical_images, annot_vertical_xmls, annot_horizontal_images, annot_horizontal_xmls):
    # are the vertical image and xml files matching
    annot_vertical = []
    for i in range(len(annot_vertical_images)):
        if annot_vertical_images[i].split(".")[0] == annot_vertical_xmls[i].split(".")[0]:
            annot_vertical.append(1)
    print(len(annot_vertical), len(annot_vertical_images))

    annot_horizontal = []
    # are the horizontally annotated images and the xml files the same?
    for j in range(len(annot_horizontal_images)):
        if annot_horizontal_images[i].split(".")[0] == annot_horizontal_xmls[i].split(".")[0]:
            annot_horizontal.append(1)
    print(len(annot_horizontal), len(annot_horizontal_images))       

In [ ]:
# now store the images as npy files?

# define a function to store the images as npy files - need two functions for vertical and horizontal separately - beacuse all images are read as horizontal images else.
def store_images_as_np_arrays_vertical(img_old_path, img_name, img_store_path):
    # join the path
    image_path = os.path.join(img_old_path, img_name)
    # read the image
    read_image = plt.imread(image_path)
    read_image = ndimage.rotate(read_image, 270)
    image_size = read_image.shape
    # show the image
    plt.imshow(read_image)
    plt.show()
    # save the image in new location
    np.save(img_store_path + '/' + img_name.split(".")[0] + '.npy', read_image)
    return(image_size)

In [ ]:
# save the outlier image in block 31 - the first vertical na image
def store_images_as_np_arrays_vertical_outlier(img_old_path, img_name, img_store_path):
    # join the path
    image_path = os.path.join(img_old_path, img_name)
    # read the image
    read_image = plt.imread(image_path)
    read_image = ndimage.rotate(read_image, 90)
    image_size = read_image.shape
    # show the image
    plt.imshow(read_image)
    plt.show()
    # save the image in new location
    np.save(img_store_path + '/' + img_name.split(".")[0] + '.npy', read_image)
    return(image_size)

In [ ]:
# define a function to store horizontal images
def store_images_as_np_arrays_horizontal(img_old_path, img_name, img_store_path):
    # join the path
    image_path = os.path.join(img_old_path, img_name)
    # read the image
    read_image = plt.imread(image_path)
    image_size = read_image.shape
    # show the image
    plt.imshow(read_image)
    plt.show()
    # save the image in new location
    np.save(img_store_path + '/' + img_name.split(".")[0] + '.npy', read_image)
    return(image_size)

In [ ]:
# note that the first horizontal image needs rotation, else the density map and the image file use for training the model will not match
def store_images_as_np_arrays_horizontal_0(img_old_path, img_name, img_store_path):
    # join the path
    image_path = os.path.join(img_old_path, img_name)
    # read the image
    read_image = plt.imread(image_path)
    read_image = ndimage.rotate(read_image, 180)
    image_size = read_image.shape
    # show the image
    plt.imshow(read_image)
    plt.show()
    # save the image in new location
    np.save(img_store_path + '/' + img_name.split(".")[0] + '.npy', read_image)
    return(image_size)

In [ ]:
# for vertically annotated images
def get_density_maps_vertical(file_name, image_path, xml_path, save_density_path):
    xml_file = file_name + '.xml'
    xml_file_path = os.path.join(xml_path, xml_file)

    # Get coords from the xml file
    # parse the xml file
    parsed_file = ET.parse(xml_file_path)
    # get the roots
    root = parsed_file.getroot()
    # get the roots here
    coords = []
    for child in root:
        for i in child:
            for j in i:
                coords.append(int(j.text))
    
    # chunk the points into sets of 4 - these are the coordinates of the bounding boxes
    points_tupples = []
    for i in range(0, len(coords), 4):
        points_tupples.append(coords[i:i + 4])

    # make a dataframe with these points
    coords_df = pd.DataFrame(points_tupples, columns = ["bleft_x", "bleft_y", "tright_x", "tright_y"])

    # compute the number of tassels in each image
    no_of_tassels = len(points_tupples)

    # compute the mid coordinates
    coords_df["mid_x"] = (round(0.5*(coords_df["bleft_x"] + coords_df["tright_x"]))).astype(int)
    coords_df["mid_y"] = (round(0.5*(coords_df["bleft_y"] + coords_df["tright_y"]))).astype(int)

    # extract the mid cordinates
    mid_coords = coords_df[["mid_x", "mid_y"]]
    warnings.filterwarnings("ignore")
    # cap the coords at the max height and width values
    mid_coords.loc[mid_coords['mid_x'] > 768, 'mid_x'] = 767
    mid_coords.loc[mid_coords['mid_y'] > 1024, 'mid_y'] = 1023
    

    # plot the bounding boxes on images
    # get image name and path
    image_name = file_name + '.npy'
    imge_file_path = os.path.join(image_path, image_name)
    # read the image
    read_image = np.load(imge_file_path)

    # check the shape of the read image
    read_image_shape = read_image.shape
    #  plot the bounding boxes on the image
    for points in points_tupples:
        annotated_image = cv2.rectangle(read_image, (points[0],points[1]), (points[2],points[3]), color = (255,0,0), thickness = 2)
    # plt.figure(figsize = (12,18))
    plt.imshow(annotated_image)
    plt.show()

    # plot the mid points on the image
    coords_list = mid_coords.values.tolist()
    # read the image again
    read_image_again = np.load(imge_file_path)
    # read_image_again = ndimage.rotate(read_image_again, 270)
    # draw the circles on image
    for i in coords_list:
        image_with_mids = cv2.circle(read_image_again, i, radius=5, color=(255, 0, 0), thickness=-1)
    # look at the annotated image
    # plt.figure(figsize = (12,18))
    plt.imshow(image_with_mids)
    plt.show()

    # also try creating the density map here
    # first create the empty maps
    np_image = np.zeros((read_image_shape[0], read_image_shape[1]))
    # get the dot maps
    for point in coords_list:
        np_image[point[1], point[0]] = 1
    # plot the image
    # plt.figure(figsize = (12,18))
    plt.imshow(np_image, cmap = "Greys")
    plt.show()

    # now define the kernel and run the convolution
    one_d_kerenel = cv2.getGaussianKernel(50,5)
    two_d_kernel = np.multiply(one_d_kerenel.T, one_d_kerenel)

    # Shape of the 2D kernel
    twoD_shape = two_d_kernel.shape
        
    # do the convolution
    convolution = ndimage.convolve(np_image, two_d_kernel)
        
    # plot the density map
    # plt.figure(figsize = (12,18))
    plt.imshow(convolution, cmap = "Greys")
    plt.show()
        
    # get the sums of the images
    img_sum = np.sum(convolution)

    # save the density map
    np.save(save_density_path + '/' + file_name + '_density_map.npy', convolution)

    return(file_name, read_image_shape, no_of_tassels, img_sum, convolution)

In [ ]:
# create density maps for the horizontally annotated images
def get_density_maps_horizontal(file_name, image_path, xml_path, save_density_path):
    xml_file = file_name + '.xml'
    xml_file_path = os.path.join(xml_path, xml_file)

    # Get coords from the xml file
    # parse the xml file
    parsed_file = ET.parse(xml_file_path)
    # get the roots
    root = parsed_file.getroot()
    # get the roots here
    coords = []
    for child in root:
        for i in child:
            for j in i:
                coords.append(int(j.text))
    
    # chunk the points into sets of 4 - these are the coordinates of the bounding boxes
    points_tupples = []
    for i in range(0, len(coords), 4):
        points_tupples.append(coords[i:i + 4])

    # make a dataframe with these points
    coords_df = pd.DataFrame(points_tupples, columns = ["bleft_x", "bleft_y", "tright_x", "tright_y"])

    # compute the number of tassels in each image
    no_of_tassels = len(points_tupples)

    # compute the mid coordinates
    coords_df["mid_x"] = (round(0.5*(coords_df["bleft_x"] + coords_df["tright_x"]))).astype(int)
    coords_df["mid_y"] = (round(0.5*(coords_df["bleft_y"] + coords_df["tright_y"]))).astype(int)

    # extract the mid cordinates
    mid_coords = coords_df[["mid_x", "mid_y"]]
    # cap the coords at the max height and width values
    mid_coords.loc[mid_coords['mid_x'] > 1024, 'mid_x'] = 1023
    mid_coords.loc[mid_coords['mid_y'] > 768, 'mid_y'] = 767
    warnings.filterwarnings("ignore")

    # plot the bounding boxes on images
    # get image name and path
    image_name = file_name + '.npy'
    imge_file_path = os.path.join(image_path, image_name)
    # read the image
    read_image = np.load(imge_file_path)
    # check the shape of the read image
    read_image_shape = read_image.shape
    #  plot the bounding boxes on the image
    for points in points_tupples:
        annotated_image = cv2.rectangle(read_image, (points[0],points[1]), (points[2],points[3]), color = (255,0,0), thickness = 2)
    # plt.figure(figsize = (12,18))
    plt.imshow(annotated_image)
    plt.show()

    # plot the mid points on the image
    coords_list = mid_coords.values.tolist()
    # read the image again
    read_image_again = np.load(imge_file_path)
    # draw the circles on image
    for i in coords_list:
        image_with_mids = cv2.circle(read_image_again, i, radius=5, color=(255, 0, 0), thickness=-1)
    # look at the annotated image
    # plt.figure(figsize = (12,18))
    plt.imshow(image_with_mids)
    plt.show()

    # also try creating the density map here
    # first create the empty maps
    np_image = np.zeros((read_image_shape[0], read_image_shape[1]))
    # get the dot maps
    for point in coords_list:
        np_image[point[1], point[0]] = 1
    # plot the image
    # plt.figure(figsize = (12,18))
    plt.imshow(np_image, cmap = "Greys")
    plt.show()

    # now define the kernel and run the convolution
    one_d_kerenel = cv2.getGaussianKernel(50,5)
    two_d_kernel = np.multiply(one_d_kerenel.T, one_d_kerenel)

    # Shape of the 2D kernel
    twoD_shape = two_d_kernel.shape
        
    # do the convolution
    convolution = ndimage.convolve(np_image, two_d_kernel)
        
    # plot the density map
    # plt.figure(figsize = (12,18))
    plt.imshow(convolution, cmap = "Greys")
    plt.show()
        
    # get the sums of the images
    img_sum = np.sum(convolution)

    # save the density map
    np.save(save_density_path + '/' + file_name + '_density_map.npy', convolution)

    return(file_name, read_image_shape, no_of_tassels, img_sum, convolution)

In [ ]:
# for vertically annotated images
def get_density_maps_not_annotated(file_name, image_path, save_density_path):
    
    # plot the bounding boxes on images
    # get image name and path
    image_name = file_name + '.npy'
    imge_file_path = os.path.join(image_path, image_name)
    # read the image
    read_image = np.load(imge_file_path)

    plt.imshow(read_image)
    plt.show()

    read_image_shape = read_image.shape
    
    # also try creating the density map here
    np_image = np.zeros((read_image_shape[0], read_image_shape[1]))

    # save the density map
    np.save(save_density_path + '/' + file_name + '_density_map.npy', np_image)

    img_sum = np.sum(np_image)

    return(file_name, read_image_shape, img_sum, np_image)

In [ ]:
def create_subwindows_and_counts(image, numpy_folder, stride = 8, kernel_size = 32):
    im_name = image.split(".")[0]
    im_file = im_name + '.npy'
    count_file = im_name + '_density_map.npy'
    # load the image and the count numpy files
    loaded_im_file = np.load(numpy_folder + '/' + im_file)
    loaded_count_file = np.load(numpy_folder + '/' + count_file)
        
    # create the subwindows and counts as follows
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]
    
    density_sums = []
    catch_image = []
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            density = loaded_count_file[i: i + kernel_size, j : j + kernel_size]
            dense_sum = np.sum(density)
            density_sums.append(dense_sum)
            sub_window = resize(sub_window, (32, 32,3))
            catch_image.append(sub_window)

    # save the combined subwindows and counts
    return(catch_image,density_sums, im_file)

##### Preprocessing for test data

Block 0202

In [ ]:
# need the images saved as npy arrays and let's also get the density counts, JIC.

In [ ]:
# list all the image and the xml files for the train data
test_contents_22 = os.listdir(block_0202)
# separate image and xml files
img_files_22, xml_files_22 = separate_img_xml(test_contents_22)
# separate annotated and not annotated images
annot_img_22, not_annot_img_22 = separate_annot_and_not(img_files_22, xml_files_22)
# separate vertical and horizontal annotated images
annot_vertical_images_22, annot_vertical_xmls_22, annot_horizontal_images_22, annot_horizontal_xmls_22 = separate_annnot_vertical_horizontal(annot_img_22, 
                                                                                                                                             xml_files_22, 8)
# check shapes
print(check_annot(annot_vertical_images_22, annot_vertical_xmls_22, annot_horizontal_images_22, annot_horizontal_xmls_22))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_22"

# blk 24
im_shapes_annot_vertical_22 = []
for item in annot_vertical_images_22:
    img_shape = store_images_as_np_arrays_vertical(block_0202, item, store_path)
    im_shapes_annot_vertical_22.append(img_shape)
print("Block 22 done!")

In [ ]:
print(im_shapes_annot_vertical_22)

In [ ]:
# store the vertically not-annotated images
store_path = "Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_22"

# blk 13
im_shapes_notannot_vertical_22 = []
for item in not_annot_img_22:
    img_shape = store_images_as_np_arrays_vertical(block_0202, item, store_path)
    im_shapes_notannot_vertical_22.append(img_shape)
print("Block 22 done!")

In [ ]:
print(im_shapes_notannot_vertical_22)

In [ ]:
# store the horizontally annotated images
store_path = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_22"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_22 = []
for item in annot_horizontal_images_22:
    img_shape = store_images_as_np_arrays_horizontal(block_0202, item, store_path)
    im_shapes_annot_horizontal_22.append(img_shape)
print("Block 22 done!")

In [ ]:
print(im_shapes_annot_horizontal_22)

In [ ]:
# xml_files_22

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'test_xml_files'
# blk 13
for file in xml_files_22:
    joined_path = os.path.join(block_0202, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_22/"
xml_path = 'test_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va_v == np.round(tasselDensities_all_va_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_test_22 = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_test_22.shape

In [ ]:
True_tasselcounts_df_vertical_annot_test_22.head()

In [ ]:
# get the outputs from this function
image_path_h = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_22"
xml_path_h = 'test_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h_v == np.round(tasselDensities_all_h_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_test_22 = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_test_22.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_test_22.head()

In [ ]:
image_path_vn = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_22'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
file_names_vn

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)
    

In [ ]:
Vertical_not_annotated_df_test_22 = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_test_22.head()

In [ ]:
Vertical_not_annotated_df_test_22.shape

In [ ]:
# stack and save the dataframe of testset true counts
test_data_22 = pd.concat((Vertical_not_annotated_df_test_22, True_tasselcounts_df_vertical_annot_test_22, True_tasselcounts_df_horizontal_annot_test_22), axis = 0)

In [ ]:
test_data_22.head()

In [ ]:
test_data_22.shape

In [ ]:
# saev this dataframe
test_data_22.to_csv("True_tassel_counts/test_data/true_test_counts_blk_22.csv", index = False)

In [ ]:
# move all the image and the xml files to a single location

va_images_dense = 'Images_and_dense_maps_npy_test/vertical_annotated_images/Block_22'
vn_images_dense = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_22'
h_images_dense = 'Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_22'

In [ ]:
shutil.copytree(va_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_22', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_22', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_22', dirs_exist_ok=True)

In [ ]:
test_files_path_22 = "Preprocessed_test_data/all_img_density_files/Block_22"

In [ ]:
test_im_and_map_contents_22 = os.listdir(test_files_path_22)

# sort these - ALWAYS sort these as the order is always messed up on HCC
test_im_and_map_contents_22.sort()

In [ ]:
len(test_im_and_map_contents_22)

In [ ]:
# get only the names of the image (npy files)
test_im_names_22 = [item for item in test_im_and_map_contents_22 if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(test_im_names_22)

In [ ]:
# test_im_names_13

In [ ]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in test_im_names_22:
    test_ims, test_maps, test_names = create_subwindows_and_counts(image, test_files_path_22, stride = 8, kernel_size = 32)
    test_im_array = np.array(test_ims)
    test_map_array = np.array(test_maps)
    print(image, test_im_array.shape, test_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_test_sub_windows_and_counts/Block_22'
    np.save(save_path + '/' + 'test_ims_' + image, test_im_array)
    # count stack
    np.save(save_path + '/' + 'test_counts_' + image, test_im_array)
    im_names.append(test_names)

Block 0205

In [ ]:
# need the images saved as npy arrays and let's also get the density counts, JIC.

In [ ]:
# list all the image and the xml files for the train data
test_contents_25 = os.listdir(block_0205)
# separate image and xml files
img_files_25, xml_files_25 = separate_img_xml(test_contents_25)
# separate annotated and not annotated images
annot_img_25, not_annot_img_25 = separate_annot_and_not(img_files_25, xml_files_25)
# separate vertical and horizontal annotated images
annot_vertical_images_25, annot_vertical_xmls_25, annot_horizontal_images_25, annot_horizontal_xmls_25 = separate_annnot_vertical_horizontal(annot_img_25, 
                                                                                                                                             xml_files_25, 6)
# check shapes
print(check_annot(annot_vertical_images_25, annot_vertical_xmls_25, annot_horizontal_images_25, annot_horizontal_xmls_25))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_25"

# blk 24
im_shapes_annot_vertical_25 = []
for item in annot_vertical_images_25:
    img_shape = store_images_as_np_arrays_vertical(block_0205, item, store_path)
    im_shapes_annot_vertical_25.append(img_shape)
print("Block 25 done!")

In [ ]:
print(im_shapes_annot_vertical_22)

In [ ]:
# store the vertically not-annotated images
store_path = "Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_25"

# blk 13
im_shapes_notannot_vertical_25 = []
for item in not_annot_img_25:
    img_shape = store_images_as_np_arrays_vertical(block_0205, item, store_path)
    im_shapes_notannot_vertical_25.append(img_shape)
print("Block 25 done!")

In [ ]:
print(im_shapes_notannot_vertical_25)

In [ ]:
# store the horizontally annotated images
store_path = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_25"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_25 = []
for item in annot_horizontal_images_25:
    img_shape = store_images_as_np_arrays_horizontal(block_0205, item, store_path)
    im_shapes_annot_horizontal_25.append(img_shape)
print("Block 25 done!")

In [ ]:
print(im_shapes_annot_horizontal_25)

In [ ]:
# xml_files_25

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'test_xml_files'
# blk 13
for file in xml_files_25:
    joined_path = os.path.join(block_0205, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_25/"
xml_path = 'test_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va_v == np.round(tasselDensities_all_va_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_test_25 = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_test_25.shape

In [ ]:
True_tasselcounts_df_vertical_annot_test_25.head()

In [ ]:
# get the outputs from this function
image_path_h = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_25"
xml_path_h = 'test_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h_v == np.round(tasselDensities_all_h_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_test_25 = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_test_25.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_test_25.head()

In [ ]:
image_path_vn = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_25'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
file_names_vn

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)
    

In [ ]:
Vertical_not_annotated_df_test_25 = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_test_25.head()

In [ ]:
Vertical_not_annotated_df_test_25.shape

In [ ]:
# stack and save the dataframe of testset true counts
test_data_25 = pd.concat((Vertical_not_annotated_df_test_25, True_tasselcounts_df_vertical_annot_test_25, True_tasselcounts_df_horizontal_annot_test_25), axis = 0)

In [ ]:
test_data_25.head()

In [ ]:
test_data_25.shape

In [ ]:
# saev this dataframe
test_data_25.to_csv("True_tassel_counts/test_data/true_test_counts_blk_25.csv", index = False)

In [ ]:
# move all the image and the xml files to a single location

va_images_dense = 'Images_and_dense_maps_npy_test/vertical_annotated_images/Block_25'
vn_images_dense = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_25'
h_images_dense = 'Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_25'

In [ ]:
shutil.copytree(va_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_25', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_25', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_25', dirs_exist_ok=True)

In [ ]:
test_files_path_25 = "Preprocessed_test_data/all_img_density_files/Block_25"

In [ ]:
test_im_and_map_contents_25 = os.listdir(test_files_path_25)

# sort these - ALWAYS sort these as the order is always messed up on HCC
test_im_and_map_contents_25.sort()

In [ ]:
len(test_im_and_map_contents_25)

In [ ]:
# get only the names of the image (npy files)
test_im_names_25 = [item for item in test_im_and_map_contents_25 if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(test_im_names_25)

In [ ]:
# test_im_names_25

In [ ]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in test_im_names_25:
    test_ims, test_maps, test_names = create_subwindows_and_counts(image, test_files_path_25, stride = 8, kernel_size = 32)
    test_im_array = np.array(test_ims)
    test_map_array = np.array(test_maps)
    print(image, test_im_array.shape, test_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_test_sub_windows_and_counts/Block_25'
    np.save(save_path + '/' + 'test_ims_' + image, test_im_array)
    # count stack
    np.save(save_path + '/' + 'test_counts_' + image, test_im_array)
    im_names.append(test_names)

Block 0206

In [ ]:
# need the images saved as npy arrays and let's also get the density counts, JIC.

In [ ]:
# list all the image and the xml files for the train data
test_contents_26 = os.listdir(block_0206)
# separate image and xml files
img_files_26, xml_files_26 = separate_img_xml(test_contents_26)
# separate annotated and not annotated images
annot_img_26, not_annot_img_26 = separate_annot_and_not(img_files_26, xml_files_26)
# separate vertical and horizontal annotated images
annot_vertical_images_26, annot_vertical_xmls_26, annot_horizontal_images_26, annot_horizontal_xmls_26 = separate_annnot_vertical_horizontal(annot_img_26, 
                                                                                                                                             xml_files_26, 7)
# check shapes
print(check_annot(annot_vertical_images_26, annot_vertical_xmls_26, annot_horizontal_images_26, annot_horizontal_xmls_26))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_26"

# blk 24
im_shapes_annot_vertical_26 = []
for item in annot_vertical_images_26:
    img_shape = store_images_as_np_arrays_vertical(block_0206, item, store_path)
    im_shapes_annot_vertical_26.append(img_shape)
print("Block 26 done!")

In [ ]:
print(im_shapes_annot_vertical_26)

In [ ]:
# store the vertically not-annotated images
store_path = "Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_26"

# blk 13
im_shapes_notannot_vertical_26 = []
for item in not_annot_img_26:
    img_shape = store_images_as_np_arrays_vertical(block_0206, item, store_path)
    im_shapes_notannot_vertical_26.append(img_shape)
print("Block 26 done!")

In [ ]:
print(im_shapes_notannot_vertical_26)

In [ ]:
# store the horizontally annotated images
store_path = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_26"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_26 = []
for item in annot_horizontal_images_26:
    img_shape = store_images_as_np_arrays_horizontal(block_0206, item, store_path)
    im_shapes_annot_horizontal_26.append(img_shape)
print("Block 26 done!")

In [ ]:
print(im_shapes_annot_horizontal_26)

In [ ]:
# xml_files_26

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'test_xml_files'
# blk 13
for file in xml_files_26:
    joined_path = os.path.join(block_0206, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_26/"
xml_path = 'test_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va_v == np.round(tasselDensities_all_va_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_test_26 = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_test_26.shape

In [ ]:
True_tasselcounts_df_vertical_annot_test_26.head()

In [ ]:
# get the outputs from this function
image_path_h = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_26"
xml_path_h = 'test_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h_v == np.round(tasselDensities_all_h_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_test_26 = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_test_26.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_test_26.head()

In [ ]:
image_path_vn = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_26'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
file_names_vn

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)
    

In [ ]:
Vertical_not_annotated_df_test_26 = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_test_26.head()

In [ ]:
Vertical_not_annotated_df_test_26.shape

In [ ]:
# stack and save the dataframe of testset true counts
test_data_26 = pd.concat((Vertical_not_annotated_df_test_26, True_tasselcounts_df_vertical_annot_test_26, True_tasselcounts_df_horizontal_annot_test_26), axis = 0)

In [ ]:
test_data_26.head()

In [ ]:
test_data_26.shape

In [ ]:
# saev this dataframe
test_data_26.to_csv("True_tassel_counts/test_data/true_test_counts_blk_26.csv", index = False)

In [ ]:
# move all the image and the xml files to a single location

va_images_dense = 'Images_and_dense_maps_npy_test/vertical_annotated_images/Block_26'
vn_images_dense = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_26'
h_images_dense = 'Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_26'

In [ ]:
shutil.copytree(va_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_26', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_26', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_26', dirs_exist_ok=True)

In [ ]:
test_files_path_26 = "Preprocessed_test_data/all_img_density_files/Block_26"

In [ ]:
test_im_and_map_contents_26 = os.listdir(test_files_path_26)

# sort these - ALWAYS sort these as the order is always messed up on HCC
test_im_and_map_contents_26.sort()

In [ ]:
len(test_im_and_map_contents_26)

In [ ]:
# get only the names of the image (npy files)
test_im_names_26 = [item for item in test_im_and_map_contents_26 if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(test_im_names_26)

In [ ]:
# test_im_names_26

In [ ]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in test_im_names_26:
    test_ims, test_maps, test_names = create_subwindows_and_counts(image, test_files_path_26, stride = 8, kernel_size = 32)
    test_im_array = np.array(test_ims)
    test_map_array = np.array(test_maps)
    print(image, test_im_array.shape, test_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_test_sub_windows_and_counts/Block_26'
    np.save(save_path + '/' + 'test_ims_' + image, test_im_array)
    # count stack
    np.save(save_path + '/' + 'test_counts_' + image, test_im_array)
    im_names.append(test_names)

Block 0302

In [ ]:
# need the images saved as npy arrays and let's also get the density counts, JIC.

In [ ]:
# list all the image and the xml files for the train data
test_contents_32 = os.listdir(block_0302)
# separate image and xml files
img_files_32, xml_files_32 = separate_img_xml(test_contents_32)
# separate annotated and not annotated images
annot_img_32, not_annot_img_32 = separate_annot_and_not(img_files_32, xml_files_32)
# separate vertical and horizontal annotated images
annot_vertical_images_32, annot_vertical_xmls_32, annot_horizontal_images_32, annot_horizontal_xmls_32 = separate_annnot_vertical_horizontal(annot_img_32, 
                                                                                                                                             xml_files_32, 6)
# check shapes
print(check_annot(annot_vertical_images_32, annot_vertical_xmls_32, annot_horizontal_images_32, annot_horizontal_xmls_32))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_32"

# blk 32
im_shapes_annot_vertical_32 = []
for item in annot_vertical_images_32:
    img_shape = store_images_as_np_arrays_vertical(block_0302, item, store_path)
    im_shapes_annot_vertical_32.append(img_shape)
print("Block 32 done!")

In [ ]:
print(im_shapes_annot_vertical_32)

In [ ]:
# store the vertically not-annotated images
store_path = "Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_32"

# blk 13
im_shapes_notannot_vertical_32 = []
for item in not_annot_img_32:
    img_shape = store_images_as_np_arrays_vertical(block_0302, item, store_path)
    im_shapes_notannot_vertical_32.append(img_shape)
print("Block 32 done!")

In [ ]:
print(im_shapes_notannot_vertical_32)

In [ ]:
# store the horizontally annotated images
store_path = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_32"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_32 = []
for item in annot_horizontal_images_32:
    img_shape = store_images_as_np_arrays_horizontal(block_0302, item, store_path)
    im_shapes_annot_horizontal_32.append(img_shape)
print("Block 32 done!")

In [ ]:
print(im_shapes_annot_horizontal_32)

In [ ]:
# xml_files_26

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'test_xml_files'
# blk 13
for file in xml_files_32:
    joined_path = os.path.join(block_0302, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_32/"
xml_path = 'test_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va_v == np.round(tasselDensities_all_va_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_test_32 = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_test_32.shape

In [ ]:
True_tasselcounts_df_vertical_annot_test_32.head()

In [ ]:
# get the outputs from this function
image_path_h = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_32"
xml_path_h = 'test_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h_v == np.round(tasselDensities_all_h_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_test_32 = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_test_32.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_test_32.head()

In [ ]:
image_path_vn = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_32'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
file_names_vn

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)
    

In [ ]:
Vertical_not_annotated_df_test_32 = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_test_32.head()

In [ ]:
Vertical_not_annotated_df_test_32.shape

In [ ]:
# stack and save the dataframe of testset true counts
test_data_32 = pd.concat((Vertical_not_annotated_df_test_32, True_tasselcounts_df_vertical_annot_test_32, True_tasselcounts_df_horizontal_annot_test_32), axis = 0)

In [ ]:
test_data_32.head()

In [ ]:
test_data_32.shape

In [ ]:
# saev this dataframe
test_data_32.to_csv("True_tassel_counts/test_data/true_test_counts_blk_32.csv", index = False)

In [ ]:
# move all the image and the xml files to a single location

va_images_dense = 'Images_and_dense_maps_npy_test/vertical_annotated_images/Block_32'
vn_images_dense = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_32'
h_images_dense = 'Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_32'

In [ ]:
shutil.copytree(va_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_32', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_32', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_32', dirs_exist_ok=True)

In [ ]:
test_files_path_32 = "Preprocessed_test_data/all_img_density_files/Block_32"

In [ ]:
test_im_and_map_contents_32 = os.listdir(test_files_path_32)

# sort these - ALWAYS sort these as the order is always messed up on HCC
test_im_and_map_contents_32.sort()

In [ ]:
len(test_im_and_map_contents_32)

In [ ]:
# get only the names of the image (npy files)
test_im_names_32 = [item for item in test_im_and_map_contents_32 if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(test_im_names_32)

In [ ]:
# test_im_names_32

In [ ]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in test_im_names_32:
    test_ims, test_maps, test_names = create_subwindows_and_counts(image, test_files_path_32, stride = 8, kernel_size = 32)
    test_im_array = np.array(test_ims)
    test_map_array = np.array(test_maps)
    print(image, test_im_array.shape, test_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_test_sub_windows_and_counts/Block_32'
    np.save(save_path + '/' + 'test_ims_' + image, test_im_array)
    # count stack
    np.save(save_path + '/' + 'test_counts_' + image, test_im_array)
    im_names.append(test_names)

In [ ]:
# test_im_names_32

Block 0303

In [ ]:
# need the images saved as npy arrays and let's also get the density counts, JIC.

In [ ]:
# list all the image and the xml files for the train data
test_contents_33 = os.listdir(block_0303)
# separate image and xml files
img_files_33, xml_files_33 = separate_img_xml(test_contents_33)
# separate annotated and not annotated images
annot_img_33, not_annot_img_33 = separate_annot_and_not(img_files_33, xml_files_33)
# separate vertical and horizontal annotated images
annot_vertical_images_33, annot_vertical_xmls_33, annot_horizontal_images_33, annot_horizontal_xmls_33 = separate_annnot_vertical_horizontal(annot_img_33, 
                                                                                                                                             xml_files_33, 6)
# check shapes
print(check_annot(annot_vertical_images_33, annot_vertical_xmls_33, annot_horizontal_images_33, annot_horizontal_xmls_33))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_33"

# blk 33
im_shapes_annot_vertical_33 = []
for item in annot_vertical_images_33:
    img_shape = store_images_as_np_arrays_vertical(block_0303, item, store_path)
    im_shapes_annot_vertical_33.append(img_shape)
print("Block 33 done!")

In [ ]:
print(im_shapes_annot_vertical_33)

In [ ]:
len(im_shapes_annot_vertical_33)

In [ ]:
# store the vertically not-annotated images
store_path = "Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_33"

# blk 33
im_shapes_notannot_vertical_33 = []
for item in not_annot_img_33:
    img_shape = store_images_as_np_arrays_vertical(block_0303, item, store_path)
    im_shapes_notannot_vertical_33.append(img_shape)
print("Block 33 done!")

In [ ]:
print(im_shapes_notannot_vertical_33)

In [ ]:
# store the horizontally annotated images
store_path = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_33"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_33 = []
for item in annot_horizontal_images_33:
    img_shape = store_images_as_np_arrays_horizontal(block_0303, item, store_path)
    im_shapes_annot_horizontal_33.append(img_shape)
print("Block 33 done!")

In [ ]:
print(im_shapes_annot_horizontal_33)

In [ ]:
# xml_files_33

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'test_xml_files'
# blk 13
for file in xml_files_33:
    joined_path = os.path.join(block_0303, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_33/"
xml_path = 'test_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va_v == np.round(tasselDensities_all_va_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_test_33 = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_test_33.shape

In [ ]:
True_tasselcounts_df_vertical_annot_test_33.head()

In [ ]:
# get the outputs from this function
image_path_h = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_33"
xml_path_h = 'test_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h_v == np.round(tasselDensities_all_h_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_test_33 = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_test_33.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_test_33.head()

In [ ]:
image_path_vn = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_33'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
file_names_vn

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)
    

In [ ]:
Vertical_not_annotated_df_test_33 = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_test_33.head()

In [ ]:
Vertical_not_annotated_df_test_33.shape

In [ ]:
# stack and save the dataframe of testset true counts
test_data_33 = pd.concat((Vertical_not_annotated_df_test_33, True_tasselcounts_df_vertical_annot_test_33, True_tasselcounts_df_horizontal_annot_test_33), axis = 0)

In [ ]:
test_data_33.head()

In [ ]:
test_data_33.shape

In [ ]:
# saev this dataframe
test_data_33.to_csv("True_tassel_counts/test_data/true_test_counts_blk_33.csv", index = False)

In [ ]:
# move all the image and the xml files to a single location

va_images_dense = 'Images_and_dense_maps_npy_test/vertical_annotated_images/Block_33'
vn_images_dense = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_33'
h_images_dense = 'Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_33'

In [ ]:
shutil.copytree(va_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_33', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_33', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_33', dirs_exist_ok=True)

In [ ]:
test_files_path_33 = "Preprocessed_test_data/all_img_density_files/Block_33"

In [ ]:
test_im_and_map_contents_33 = os.listdir(test_files_path_33)

# sort these - ALWAYS sort these as the order is always messed up on HCC
test_im_and_map_contents_33.sort()

In [ ]:
len(test_im_and_map_contents_33)

In [ ]:
# get only the names of the image (npy files)
test_im_names_33 = [item for item in test_im_and_map_contents_33 if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(test_im_names_33)

In [ ]:
# test_im_names_33

In [ ]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in test_im_names_33:
    test_ims, test_maps, test_names = create_subwindows_and_counts(image, test_files_path_33, stride = 8, kernel_size = 32)
    test_im_array = np.array(test_ims)
    test_map_array = np.array(test_maps)
    print(image, test_im_array.shape, test_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_test_sub_windows_and_counts/Block_33'
    np.save(save_path + '/' + 'test_ims_' + image, test_im_array)
    # count stack
    np.save(save_path + '/' + 'test_counts_' + image, test_im_array)
    im_names.append(test_names)

Block 0304

In [ ]:
# need the images saved as npy arrays and let's also get the density counts, JIC.

In [ ]:
# list all the image and the xml files for the train data
test_contents_34 = os.listdir(block_0304)
# separate image and xml files
img_files_34, xml_files_34 = separate_img_xml(test_contents_34)
# separate annotated and not annotated images
annot_img_34, not_annot_img_34 = separate_annot_and_not(img_files_34, xml_files_34)
# separate vertical and horizontal annotated images
annot_vertical_images_34, annot_vertical_xmls_34, annot_horizontal_images_34, annot_horizontal_xmls_34 = separate_annnot_vertical_horizontal(annot_img_34, 
                                                                                                                                             xml_files_34, 6)
# check shapes
print(check_annot(annot_vertical_images_34, annot_vertical_xmls_34, annot_horizontal_images_34, annot_horizontal_xmls_34))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_34"

# blk 34
im_shapes_annot_vertical_34 = []
for item in annot_vertical_images_34:
    img_shape = store_images_as_np_arrays_vertical(block_0304, item, store_path)
    im_shapes_annot_vertical_34.append(img_shape)
print("Block 34 done!")

In [ ]:
print(im_shapes_annot_vertical_34)

In [ ]:
len(im_shapes_annot_vertical_34)

In [ ]:
# store the vertically not-annotated images
store_path = "Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_34"

# blk 33
im_shapes_notannot_vertical_34 = []
for item in not_annot_img_34:
    img_shape = store_images_as_np_arrays_vertical(block_0304, item, store_path)
    im_shapes_notannot_vertical_34.append(img_shape)
print("Block 34 done!")

In [ ]:
print(im_shapes_notannot_vertical_34)

In [ ]:
# store the horizontally annotated images
store_path = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_34"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_34 = []
for item in annot_horizontal_images_34:
    img_shape = store_images_as_np_arrays_horizontal(block_0304, item, store_path)
    im_shapes_annot_horizontal_34.append(img_shape)
print("Block 34 done!")

In [ ]:
print(im_shapes_annot_horizontal_34)

In [ ]:
# xml_files_34

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'test_xml_files'
# blk 13
for file in xml_files_34:
    joined_path = os.path.join(block_0304, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_34/"
xml_path = 'test_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va_v == np.round(tasselDensities_all_va_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_test_34 = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_test_34.shape

In [ ]:
True_tasselcounts_df_vertical_annot_test_34.head()

In [ ]:
# get the outputs from this function
image_path_h = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_34"
xml_path_h = 'test_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h_v == np.round(tasselDensities_all_h_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_test_34 = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_test_34.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_test_34.head()

In [ ]:
image_path_vn = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_34'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
file_names_vn

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)
    

In [ ]:
Vertical_not_annotated_df_test_34 = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_test_34.head()

In [ ]:
Vertical_not_annotated_df_test_34.shape

In [ ]:
# stack and save the dataframe of testset true counts
test_data_34 = pd.concat((Vertical_not_annotated_df_test_34, True_tasselcounts_df_vertical_annot_test_34, True_tasselcounts_df_horizontal_annot_test_34), axis = 0)

In [ ]:
test_data_34.head()

In [ ]:
test_data_34.shape

In [ ]:
# saev this dataframe
test_data_34.to_csv("True_tassel_counts/test_data/true_test_counts_blk_34.csv", index = False)

In [ ]:
# move all the image and the xml files to a single location

va_images_dense = 'Images_and_dense_maps_npy_test/vertical_annotated_images/Block_34'
vn_images_dense = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_34'
h_images_dense = 'Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_34'

In [ ]:
shutil.copytree(va_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_34', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_34', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_34', dirs_exist_ok=True)

In [ ]:
test_files_path_34 = "Preprocessed_test_data/all_img_density_files/Block_34"

In [ ]:
test_im_and_map_contents_34 = os.listdir(test_files_path_34)

# sort these - ALWAYS sort these as the order is always messed up on HCC
test_im_and_map_contents_34.sort()

In [ ]:
len(test_im_and_map_contents_34)

In [ ]:
# get only the names of the image (npy files)
test_im_names_34 = [item for item in test_im_and_map_contents_34 if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(test_im_names_34)

In [ ]:
# test_im_names_34

In [ ]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in test_im_names_34:
    test_ims, test_maps, test_names = create_subwindows_and_counts(image, test_files_path_34, stride = 8, kernel_size = 32)
    test_im_array = np.array(test_ims)
    test_map_array = np.array(test_maps)
    print(image, test_im_array.shape, test_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_test_sub_windows_and_counts/Block_34'
    np.save(save_path + '/' + 'test_ims_' + image, test_im_array)
    # count stack
    np.save(save_path + '/' + 'test_counts_' + image, test_im_array)
    im_names.append(test_names)

Block 0305

In [ ]:
# need the images saved as npy arrays and let's also get the density counts, JIC.

In [ ]:
# list all the image and the xml files for the train data
test_contents_35 = os.listdir(block_0305)
# separate image and xml files
img_files_35, xml_files_35 = separate_img_xml(test_contents_35)
# separate annotated and not annotated images
annot_img_35, not_annot_img_35 = separate_annot_and_not(img_files_35, xml_files_35)
# separate vertical and horizontal annotated images
annot_vertical_images_35, annot_vertical_xmls_35, annot_horizontal_images_35, annot_horizontal_xmls_35 = separate_annnot_vertical_horizontal(annot_img_35, 
                                                                                                                                             xml_files_35, 6)
# check shapes
print(check_annot(annot_vertical_images_35, annot_vertical_xmls_35, annot_horizontal_images_35, annot_horizontal_xmls_35))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_35"

# blk 35
im_shapes_annot_vertical_35 = []
for item in annot_vertical_images_35:
    img_shape = store_images_as_np_arrays_vertical(block_0305, item, store_path)
    im_shapes_annot_vertical_35.append(img_shape)
print("Block 35 done!")

In [ ]:
print(im_shapes_annot_vertical_35)

In [ ]:
len(im_shapes_annot_vertical_35)

In [ ]:
# store the vertically not-annotated images
store_path = "Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_35"

# blk 35
im_shapes_notannot_vertical_35 = []
for item in not_annot_img_35:
    img_shape = store_images_as_np_arrays_vertical(block_0305, item, store_path)
    im_shapes_notannot_vertical_35.append(img_shape)
print("Block 35 done!")

In [ ]:
print(im_shapes_notannot_vertical_35)

In [ ]:
# store the horizontally annotated images
store_path = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_35"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_35 = []
for item in annot_horizontal_images_35:
    img_shape = store_images_as_np_arrays_horizontal(block_0305, item, store_path)
    im_shapes_annot_horizontal_35.append(img_shape)
print("Block 35 done!")

In [ ]:
print(im_shapes_annot_horizontal_35)

In [ ]:
# xml_files_35

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'test_xml_files'
# blk 13
for file in xml_files_35:
    joined_path = os.path.join(block_0305, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_35/"
xml_path = 'test_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va_v == np.round(tasselDensities_all_va_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_test_35 = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_test_35.shape

In [ ]:
True_tasselcounts_df_vertical_annot_test_35.head()

In [ ]:
# get the outputs from this function
image_path_h = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_35"
xml_path_h = 'test_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h_v == np.round(tasselDensities_all_h_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_test_35 = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_test_35.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_test_35.head()

In [ ]:
image_path_vn = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_35'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
file_names_vn

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)
    

In [ ]:
Vertical_not_annotated_df_test_35 = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_test_35.head()

In [ ]:
Vertical_not_annotated_df_test_35.shape

In [ ]:
# stack and save the dataframe of testset true counts
test_data_35 = pd.concat((Vertical_not_annotated_df_test_35, True_tasselcounts_df_vertical_annot_test_35, True_tasselcounts_df_horizontal_annot_test_35), axis = 0)

In [ ]:
test_data_35.head()

In [ ]:
test_data_35.shape

In [ ]:
# saev this dataframe
test_data_35.to_csv("True_tassel_counts/test_data/true_test_counts_blk_35.csv", index = False)

In [ ]:
# move all the image and the xml files to a single location

va_images_dense = 'Images_and_dense_maps_npy_test/vertical_annotated_images/Block_35'
vn_images_dense = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_35'
h_images_dense = 'Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_35'

In [ ]:
shutil.copytree(va_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_35', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_35', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_35', dirs_exist_ok=True)

In [ ]:
test_files_path_35 = "Preprocessed_test_data/all_img_density_files/Block_35"

In [ ]:
test_im_and_map_contents_35 = os.listdir(test_files_path_35)

# sort these - ALWAYS sort these as the order is always messed up on HCC
test_im_and_map_contents_35.sort()

In [ ]:
len(test_im_and_map_contents_35)

In [ ]:
# get only the names of the image (npy files)
test_im_names_35 = [item for item in test_im_and_map_contents_35 if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(test_im_names_35)

In [ ]:
# test_im_names_34

In [ ]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in test_im_names_35:
    test_ims, test_maps, test_names = create_subwindows_and_counts(image, test_files_path_35, stride = 8, kernel_size = 32)
    test_im_array = np.array(test_ims)
    test_map_array = np.array(test_maps)
    print(image, test_im_array.shape, test_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_test_sub_windows_and_counts/Block_35'
    np.save(save_path + '/' + 'test_ims_' + image, test_im_array)
    # count stack
    np.save(save_path + '/' + 'test_counts_' + image, test_im_array)
    im_names.append(test_names)

Block 0306

In [ ]:
# need the images saved as npy arrays and let's also get the density counts, JIC.

In [ ]:
# list all the image and the xml files for the train data
test_contents_36 = os.listdir(block_0306)
# separate image and xml files
img_files_36, xml_files_36 = separate_img_xml(test_contents_36)
# separate annotated and not annotated images
annot_img_36, not_annot_img_36 = separate_annot_and_not(img_files_36, xml_files_36)
# separate vertical and horizontal annotated images
annot_vertical_images_36, annot_vertical_xmls_36, annot_horizontal_images_36, annot_horizontal_xmls_36 = separate_annnot_vertical_horizontal(annot_img_36, 
                                                                                                                                             xml_files_36, 6)
# check shapes
print(check_annot(annot_vertical_images_36, annot_vertical_xmls_36, annot_horizontal_images_36, annot_horizontal_xmls_36))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_36"

# blk 36
im_shapes_annot_vertical_36 = []
for item in annot_vertical_images_36:
    img_shape = store_images_as_np_arrays_vertical(block_0306, item, store_path)
    im_shapes_annot_vertical_36.append(img_shape)
print("Block 36 done!")

In [ ]:
print(im_shapes_annot_vertical_36)

In [ ]:
len(im_shapes_annot_vertical_36)

In [ ]:
# store the vertically not-annotated images
store_path = "Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_36"

# blk 36
im_shapes_notannot_vertical_36 = []
for item in not_annot_img_36:
    img_shape = store_images_as_np_arrays_vertical(block_0306, item, store_path)
    im_shapes_notannot_vertical_36.append(img_shape)
print("Block 36 done!")

In [ ]:
print(im_shapes_notannot_vertical_36)

In [ ]:
# store the horizontally annotated images
store_path = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_36"

im_shapes_annot_horizontal_36 = []
for item in annot_horizontal_images_36:
    img_shape = store_images_as_np_arrays_horizontal(block_0306, item, store_path)
    im_shapes_annot_horizontal_36.append(img_shape)
print("Block 36 done!")

In [ ]:
print(im_shapes_annot_horizontal_36)

In [ ]:
# xml_files_36

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'test_xml_files'
# blk 13
for file in xml_files_36:
    joined_path = os.path.join(block_0306, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Images_and_dense_maps_npy_test/vertical_annotated_images/Block_36/"
xml_path = 'test_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va_v == np.round(tasselDensities_all_va_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_test_36 = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_test_36.shape

In [ ]:
True_tasselcounts_df_vertical_annot_test_36.head()

In [ ]:
# get the outputs from this function
image_path_h = "Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_36"
xml_path_h = 'test_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h_v == np.round(tasselDensities_all_h_v))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_test_36 = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_test_36.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_test_36.head()

In [ ]:
image_path_vn = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_36'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
file_names_vn

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)
    

In [ ]:
Vertical_not_annotated_df_test_36 = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_test_36.head()

In [ ]:
Vertical_not_annotated_df_test_36.shape

In [ ]:
# stack and save the dataframe of testset true counts
test_data_36 = pd.concat((Vertical_not_annotated_df_test_36, True_tasselcounts_df_vertical_annot_test_36, True_tasselcounts_df_horizontal_annot_test_36), axis = 0)

In [ ]:
test_data_36.head()

In [ ]:
test_data_36.shape

In [ ]:
# saev this dataframe
test_data_36.to_csv("True_tassel_counts/test_data/true_test_counts_blk_36.csv", index = False)

In [ ]:
# move all the image and the xml files to a single location

va_images_dense = 'Images_and_dense_maps_npy_test/vertical_annotated_images/Block_36'
vn_images_dense = 'Images_and_dense_maps_npy_test/vertical_not_annotated_images/Block_36'
h_images_dense = 'Images_and_dense_maps_npy_test/horizontal_annotated_images/Block_36'

In [ ]:
shutil.copytree(va_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_36', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_36', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Preprocessed_test_data/all_img_density_files/Block_36', dirs_exist_ok=True)

In [ ]:
test_files_path_36 = "Preprocessed_test_data/all_img_density_files/Block_36"

In [ ]:
test_im_and_map_contents_36 = os.listdir(test_files_path_36)

# sort these - ALWAYS sort these as the order is always messed up on HCC
test_im_and_map_contents_36.sort()

In [ ]:
len(test_im_and_map_contents_36)

In [ ]:
# get only the names of the image (npy files)
test_im_names_36 = [item for item in test_im_and_map_contents_36 if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(test_im_names_36)

In [ ]:
# test_im_names_36

In [ ]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in test_im_names_36:
    test_ims, test_maps, test_names = create_subwindows_and_counts(image, test_files_path_36, stride = 8, kernel_size = 32)
    test_im_array = np.array(test_ims)
    test_map_array = np.array(test_maps)
    print(image, test_im_array.shape, test_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_test_sub_windows_and_counts/Block_36'
    np.save(save_path + '/' + 'test_ims_' + image, test_im_array)
    # count stack
    np.save(save_path + '/' + 'test_counts_' + image, test_im_array)
    im_names.append(test_names)